In [2]:
from tools.annotation_bbox import clip_dataset_bbox_xyxy
from tools.extract_frames import extract_frames
from tools.annotation_converter import viame2standard
import pandas as pd
import json
import shutil
import os
import cv2

## 1. Download the data

In [2]:
data = 'viame_sp'
output_dir = "/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/"
output_path = f"{output_dir}{data}.zip"
download_link = "https://viame.kitware.com/api/v1/dive_dataset/export?folderIds=[%2265c39a4ea8df0635ce3b443e%22,%2265c6378785591d30c22fbe9e%22,%2265b7e3e1a4c218785d460e30%22,%2265c5a73263ff94bc31466fd9%22,%2265c5a779b3a7cddd61cf098d%22,%2265b90381a4c218785d462001%22,%2265baad76cf5a99794eaf0791%22,%2265baae0bd17f7c8fa5bf73b5%22,%2265baaef8a4c218785d463051%22,%2265c3a78a8cb22dc59f13f1af%22,%2265c3fb40ecb4b390ccc31d41%22,%2265c3fc7585591d30c22f64de%22,%2265c5a7148cb22dc59f144e17%22]"

In [ ]:
!wget -O {output_path} {download_link}
!unzip {output_path} -d {output_dir}

## 2. Construct dataset
Some data is image sequence, so we already have the frames. Others are video sequences, so we need to extract them.


Therefore, have a ANNOTATIONS FIRST approach:
1. Extract the annotations
2. From the annotations select the frames you need

In [25]:
download_dir = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads'

In [30]:
def get_annotation_fps(image_sequence_path):
  metadata_path = os.path.join(image_sequence_path, 'meta.json')
  with open(metadata_path) as f:
    metadata = json.load(f)
  return int(metadata['fps'])

In [3]:
phase3_path = '/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3'

def process_image_sequence(image_sequence_path):
    source = os.path.basename(image_sequence_path)
    source_output_path = os.path.join(phase3_path, source)
    os.makedirs(source_output_path)

    try:
        # Convert annotations
        fps = get_annotation_fps(image_sequence_path)
        print(f"Processing {source} with fps {fps}...")
        viame_annotations_path = os.path.join(image_sequence_path, 'annotations.viame.csv')
        sharktrack_annotations = viame2standard(viame_annotations_path, source, download_dir, fps, is_video=False)
        sharktrack_annotations.to_csv(os.path.join(source_output_path, 'annotations.csv'), index=False)

        # Move frames
        for filename in os.listdir(image_sequence_path):
            if filename.endswith('.jpg'):
                curr_path = os.path.join(image_sequence_path, filename)
                if filename.startswith(source):
                    print(f"Filename {filename} already starts with source {source}")
                    out_path = os.path.join(source_output_path, filename)
                else:
                    out_path = os.path.join(source_output_path, f"{source}_{filename}")
                shutil.copy(curr_path, out_path)
    except Exception as e:
        os.rmdir(source_output_path)

In [44]:
import traceback

def process_video_annotations(source_path):
  source = os.path.basename(source_path)
  source_output_path = os.path.join(phase3_path, source)
  os.makedirs(source_output_path)

  fps = get_annotation_fps(source_path)
  print(f"Processing {source} with fps {fps}...")

  try:
    # Extract frames from video
    video_name = next(f for f in os.listdir(source_path) if '.transcoded' in f)
    video_path = os.path.join(source_path, video_name)
    desired_fps = 1
    extract_frames(video_path, source_path, source, desired_fps)

    # Convert annotations
    viame_annotations_path = os.path.join(source_path, 'annotations.viame.csv')
    sharktrack_annotations = viame2standard(viame_annotations_path, source, download_dir, fps, is_video=True)
    sharktrack_annotations.to_csv(os.path.join(source_output_path, 'annotations.csv'), index=False)

    # Move frames
    for filename in os.listdir(source_path):
      if filename.endswith('.jpg'):
        assert filename.startswith(f"{source}_frame"), f"Filename {filename} does not start with source {source}"
        curr_path = os.path.join(source_path, filename)
        out_path = os.path.join(source_output_path, filename)
        shutil.copy(curr_path, out_path)
  except Exception as e:
    traceback.print_exc()
    os.rmdir(source_output_path)
    for filename in os.listdir(source_path):
      if filename.endswith('.jpg'):
        os.remove(os.path.join(source_path, filename))

In [45]:
# If image: just move all the frames however they are named
# If video AND 1fps (sp), extract the frame at 1fps AND map it to annotation frame id
for source in os.listdir(download_dir):
  source_path = os.path.join(download_dir, source)
  if not os.path.isdir(source_path):
    continue
  
  source_is_img_sequence = any(file.endswith(".jpg") for file in os.listdir(source_path))

  if source_is_img_sequence:
    process_image_sequence(source_path)
  else:
    process_video_annotations(source_path)

Processing val1_medium1 with fps 10...
Extracted 20 frames from /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3_dev/downloads/val1_medium1/medium1.transcoded.mp4


## 3. Clean BBoxes

In [4]:
clip_dataset_bbox_xyxy(phase3_path)

Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/shlife_bull7/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/sp_natgeo1/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/gfp_rand9/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/gfp_rand3/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/shlife_scalloped4/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/gfp_caicos1/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/gfp_kiribati1/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/gfp_belize1/annotations.csv...
Clipping bbox xyxy of /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/phase3/shlife_scalloped3/annotations.csv...


## 4. Visualise Dataset and check everything is alright

## 5. Remove dumb FP